In [20]:
import pandas as pd


import ssl

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import nltk
from src.WearType import WearType

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

#nltk.download()  # uncomment this line if you execute this notebook the first time, afterwards you can comment it again

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

df = pd.read_parquet('../../datasets/cleaned/polyvore_v1.parquet')
df['wearType'] = df['wearType'].apply(lambda x: WearType(x))
df

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/romanzberg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/romanzberg/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/romanzberg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/romanzberg/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,outfitName,outfitViews,wearType,type,outfitLikes,date,set_id,outfitDesc,clothingItemIndex,clothingItemName,clothingItemPrice,clothingItemLikes,clothingItemCategoryId,imagePath
7,Being a Vans shoe model with Luke. Idk about t...,188,2,top,9,Two years,120161271,A fashion look from April 2014 featuring destr...,1,nirvana distressed t-shirt,10.0,1290,21,raw/images/120161271/1.jpg
8,Being a Vans shoe model with Luke. Idk about t...,188,4,pants,9,Two years,120161271,A fashion look from April 2014 featuring destr...,2,rag bone rock w/ black skinny jeans,235.0,17406,237,raw/images/120161271/2.jpg
15,These Chanel bags is a bad habit .x,562,2,top,32,Two years,143656996,12.19.14,1,monki singlet,16.0,20094,104,raw/images/143656996/1.jpg
17,These Chanel bags is a bad habit .x,562,4,pants,32,Two years,143656996,12.19.14,3,topshop moto joni high rise skinny jeans,65.0,23324,237,raw/images/143656996/3.jpg
19,These Chanel bags is a bad habit .x,562,1,bag,32,Two years,143656996,12.19.14,5,pre-owned chanel shoulder bag,2450.0,4489,37,raw/images/143656996/5.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142460,Army,281,3,outwear,30,One year,187953065,A fashion look from January 2016 by catherine-...,2,yoins army green buttoned slit lapel coat,33.0,431,24,raw/images/187953065/2.jpg
142461,Army,281,4,skirt,30,One year,187953065,A fashion look from January 2016 by catherine-...,3,topshop wool blend a-line skirt,19.0,1456,9,raw/images/187953065/3.jpg
142462,Army,281,5,shoes,30,One year,187953065,A fashion look from January 2016 by catherine-...,4,ralph lauren collection canvas sandals,230.0,831,46,raw/images/187953065/4.jpg
142463,Army,281,1,bag,30,One year,187953065,A fashion look from January 2016 by catherine-...,5,alexander mcqueen hunter green black mini leat...,1010.0,16,37,raw/images/187953065/5.jpg


In [16]:
stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words('english'))

df_text_analisys = df[['set_id', 'outfitName', 'outfitDesc', 'outfitLikes']]
df_text_analisys = df_text_analisys[['set_id', 'outfitName', 'outfitDesc', 'outfitLikes']].drop_duplicates()

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))


def normalize(text):
    tokens = nltk.word_tokenize(text)
    result = [lemmatize_stemming(token) for token in tokens
              if token not in stop_words and len(token) > 3]
    return result


names = df_text_analisys["outfitName"].apply(lambda x: normalize(x))
desc = df_text_analisys['outfitDesc'].apply(lambda x: normalize(x))
#df_only_valid_outfits

In [17]:
names = pd.DataFrame(names)
names.explode('outfitName').value_counts()

outfitName   
style            685
school           364
street           351
skirt            309
casual           303
                ... 
fiddlingabout      1
field              1
fierc              1
fifti              1
😙😙😙😘😚😗             1
Name: count, Length: 3404, dtype: int64

In [6]:
desc = pd.DataFrame(desc)
desc.explode('outfitDesc').value_counts()

outfitDesc   
look             6792
featur           5945
fashion          5912
2016             2659
2017             1431
                 ... 
hoschek             1
horsevespalov       1
horror-hotti        1
horizon             1
😱😍❤️❤️❤️❤️          1
Name: count, Length: 9136, dtype: int64

In [7]:
df_text_analisys

,set_id,outfitName,outfitDesc,outfitLikes
7,120161271,Being a Vans shoe model with Luke. Idk about t...,A fashion look from April 2014 featuring destr...,9
15,143656996,These Chanel bags is a bad habit .x,12.19.14,32
43,216470135,the people i choose never choose me and my hea...,A fashion look from February 2017 by exco feat...,395
50,216220312,Be happy!,A fashion look from February 2017 by rasa-j fe...,233
85,192203629,yellow grey,A fashion look from February 2016 by lifestyle...,153
...,...,...,...,...
142396,201717504,Denim Shorts (Outfit Only),A fashion look from June 2016 featuring blue t...,367
142406,216589548,winter coat,A fashion look from February 2017 by emcf3548 ...,116
142418,216860218,Outfit of the Day,A fashion look from February 2017 by dressedby...,214
142432,211099953,park and coffee,A fashion look from November 2016 by dance-nat...,40


In [8]:
print(df_text_analisys['set_id'].unique().shape[0])
print(df_text_analisys.shape[0])

6793
6793


In [9]:
df_text_analisys['outfitLikes'].describe()

count    6793.000000
mean      354.551008
std       567.661429
min         2.000000
25%        28.000000
50%       134.000000
75%       428.000000
max      7294.000000
Name: outfitLikes, dtype: float64